In [ ]:
%pip install -U langchain jq

In [ ]:
from langchain.document_loaders import JSONLoader
from pprint import pprint

loader = JSONLoader(
    file_path='./monster_text.json', jq_schema='.Ancient Deep Crow-AI[].text')
data = loader.load()

